This notebook presents the techniques of displaying the precision of the Radio Telemetry Tracker system.

In [60]:
import numpy as np
import matplotlib.pyplot as plt; plt.ion()
from scipy.optimize import least_squares

# Model Estimation
To determine the location of the radio transmitter, we need to find a relationship between the measurement location and the radio amplitude measurement.  We use the radio signal path loss model: $L = 10nlog_{10}(d)$.

We can then rewrite this in terms of receive and transmit power, $R$ and $P$: $R = P - 10nlog_{10}(d) + C$, where $C$ represents receive losses not due to path loss.

In theory, $n$ is 2 for path loss in a vacuum, but in practice, we see $n$ range from $2$ to $6$.

However, we cannot in principle differentiate between P and C, as they are summed together, so we can simply combine them into a single parameter $k$: $R = k - 10nlog_{10}(d)$

In [61]:
def receivePowerModel(d, k, n):
    return k - 10 * n * np.log10(d)

When we receive data, we can extract the following information: drone location in x, y, and z, and transmitter receive power.  We can then put this information into a matrix $D = \begin{bmatrix}x_d & y_d & z_d & R\end{bmatrix}$, where $x_d$ is the column vector of drone x coordinates, $y_d$ is the column vector of drone y coordinates, $z_d$ is the column vector of drone z coordinates, and $R$ is the column vector of receive powers.  We generate a simulated dataset below:

In [62]:
vx = np.array([[5, 1, 0],
               [-1, 5, 0],
               [-5, -1, 0],
               [1, -5, 0]])
tx = (477954, 3638577, 11, 'S')
txp = 100
n = 4
C = 1
d = 1000
tx_loc = np.array([tx[0], tx[1], 0])
dx_loc = np.array([tx[0] - 100, tx[1], 30])
s_per_leg = 30
c_loc = dx_loc
D_p = []
for leg in range(4):
    for s in range(s_per_leg):
        c_loc += vx[leg,:]
        dist = np.linalg.norm(tx_loc - c_loc)
        R = receivePowerModel(dist, txp - C, n)
        D_p.append([c_loc[0], c_loc[1], c_loc[2], R])
simulated_D = np.array(D_p)

To localize the transmitter, we simply take the measurements in $D$ and fit them to the model in `receivePowerModel`

In [68]:
def residuals(params, data):
    tx = params[0]
    ty = params[1]
    t_loc = np.array([tx, ty, 0])
    
    k = params[2]
    n = params[3]
    
    R = data[:,3]
    d_loc = data[:,0:3]
    
    residuals = np.zeros(len(R))
    for i in range(len(R)):
        residuals[i] = R[i] - receivePowerModel(np.linalg.norm(t_loc - d_loc[i,:]), k, n)
    return residuals

initialGuess = np.array([np.mean(simulated_D[:,0]), np.mean(simulated_D[:,1]),
                         np.max(simulated_D[:,3]), 4])
results = least_squares(residuals, initialGuess, kwargs={'data':simulated_D})
print(results.x)
print("Lateral error: %f" % np.linalg.norm(tx_loc - np.array([results.x[0], results.x[1], 0])))
print("k error: %f" % (txp - C - results.x[2]))
print("n error: %f" % (n - results.x[3]))

[4.77954000e+05 3.63857700e+06 9.89999992e+01 3.99999996e+00]
Lateral error: 0.000001
k error: 0.000001
n error: 0.000000


As shown above, the estimated result is nearly identical to the simulated parameters, which should be the case, as the only source of noise here is from quantization error.

In order to accurately simulate the data from the drone, we need to add some error to the measurements.  In this case, we will add Gaussian noise to each measurement.  For the $x$ and $y$ coordinates, we will add Gaussian noise with $\mu = 0$ and $\sigma^2 = 3$.  For the $z$ coordinate, we will add Gaussian noise with $\mu = 0$ and $\sigma^2 = 0.5$.  For the $R$ measurement, we will add Gaussian noise with $\mu = 0$ and $\sigma^2 = 5$.

In [67]:
c_loc = dx_loc
D_p = []
s_xy = np.sqrt(3)
s_z = np.sqrt(0.5)
s_R = np.sqrt(5)
for leg in range(4):
    for s in range(s_per_leg):
        c_loc += vx[leg,:]
        dist = np.linalg.norm(tx_loc - c_loc)
        R = receivePowerModel(dist, txp - C, n)
        D_p.append([c_loc[0] + np.random.normal(0,s_xy),
                    c_loc[1] + np.random.normal(0,s_xy),
                    c_loc[2] + np.random.normal(0,s_z),
                    R + np.random.normal(0,s_R)])
simulated_D_error = np.array(D_p)
print(simulated_D_error[0:5,:])

[[4.77859325e+05 3.63857800e+06 3.02344649e+01 2.16459911e+01]
 [4.77863334e+05 3.63857832e+06 3.08992596e+01 2.10609051e+01]
 [4.77868570e+05 3.63857970e+06 3.00403799e+01 2.51313227e+01]
 [4.77874206e+05 3.63857989e+06 2.91945001e+01 1.61325356e+01]
 [4.77879148e+05 3.63858156e+06 3.13579579e+01 1.83828436e+01]]


In [65]:
initialGuess = np.array([np.mean(simulated_D_error[:,0]), np.mean(simulated_D_error[:,1]),
                         np.max(simulated_D_error[:,3]), 4])
results = least_squares(residuals, initialGuess, kwargs={'data':simulated_D_error})
print(results.x)
print("Lateral error: %f" % np.linalg.norm(tx_loc - np.array([results.x[0], results.x[1], 0])))
print("k error: %f" % (txp - C - results.x[2]))
print("n error: %f" % (n - results.x[3]))

[4.77955595e+05 3.63857199e+06 1.02789290e+02 4.15622580e+00]
Lateral error: 5.257425
k error: -3.789290
n error: -0.156226


# Precision Visualization

In order to provide a metric of how certain our estimate is, we need to provide a way to visualize the precision of our estimate.